In [ ]:
import sys
import numpy as np
import cv2
from sklearn.decomposition import PCA, IncrementalPCA

from matplotlib import pyplot as plt

import tables
from tables import IsDescription, StringCol, Float32Col, Float64Col
import glob
# import uuid

# Global descriptor

In [ ]:
def edge_distribution(img, blocks, nbins):
    edges = cv2.Canny(img, threshold1=50, threshold2=120)
    
    smoothed = cv2.GaussianBlur(gray_img, ksize=(3,3), sigmaX=0, sigmaY=0)
    dx = cv2.Sobel(smoothed, cv2.CV_16S, ksize=3, dx=1, dy=0)
    dy = cv2.Sobel(smoothed, cv2.CV_32F, ksize=3, dx=0, dy=1)
    theta = np.arctan2(dy, dx)
    
    ysize, xsize = img.shape
    yblocks, xblocks = blocks
    xstep = math.ceil(xsize / xblocks)
    ystep = math.ceil(ysize / yblocks)
    if xstep*(xblocks-1) >= xsize or ystep*(yblocks-1) >= ysize:
        raise Exception('The image of shape %s is not suitable for blocks of %s.' % (img.shape, blocks))

    masks = []
    for i in range(xblocks):
        x = np.zeros(xsize)
        x[i*xstep:min((i+1)*xstep, xsize)] = 1
        for j in range(yblocks):
            y = np.zeros(ysize)
            y[j*ystep:min((j+1)*ystep, ysize)] = 1

            mask = np.outer(y, x)
            masks += [mask]
    
    counters = []
    bins = np.append(np.arange(-np.pi, np.pi, 2*np.pi/nbins), np.pi)
    for b in range(len(masks)):
        for i in range(nbins):
            theta_mask = (theta >= bins[i]) & (theta < bins[i+1])
            edges0 = theta_mask * masks[b] * edges
            counters += [np.sum(edges0)]
            
        counters += [np.sum(masks[b] * edges) / np.sum(edges) if np.sum(edges) > 0 else 0]
        
    return counters

def gray_averages(img, blocks):
    ysize, xsize = img.shape
    yblocks, xblocks = blocks
    xstep = math.ceil(xsize / xblocks)
    ystep = math.ceil(ysize / yblocks)
    if xstep*(xblocks-1) >= xsize or ystep*(yblocks-1) >= ysize:
        raise Exception('The image of shape %s is not suitable for blocks of %s.' % (img.shape, blocks))

    masks = []
    for i in range(xblocks):
        x = np.zeros(xsize)
        x[i*xstep:min((i+1)*xstep, xsize)] = 1
        for j in range(yblocks):
            y = np.zeros(ysize)
            y[j*ystep:min((j+1)*ystep, ysize)] = 1

            mask = np.outer(y, x)
            masks += [mask]
            
    avg = []
    for b in range(len(masks)):
        avg += [np.sum(gray_img * masks[b]) / np.sum(masks[b])]
        
    return avg

In [ ]:
img_list = glob.glob('./**/*.jpg', recursive=True)[:10000]

In [ ]:
row = img_glob_feat_table.row
for p in img_list:
    i = p.rfind('/')
    h = p[i+1:]
#     if len(h) != 32:
#         print('Not good file.')
#         continue
    gray_img = cv2.cvtColor(cv2.imread(p), cv2.COLOR_BGR2GRAY)
    feat0 = edge_distribution(gray_img, (2,2), 12)
    feat1 = gray_averages(gray_img, (8,8))
    feat = feat0 + feat1
    
    row['hash'] = h
    row['glob_feat'] = feat
    row.append()
    
img_glob_feat_table.flush()
raw_feat_db.close()

# Produce raw global features and fit PCA

In [ ]:
class RawGlobFeature(IsDescription):
    hash = StringCol(48)
    glob_feat  = Float64Col(116)

feat_db = tables.open_file("./dup_img_feat.h5", mode="a", title="Features for duplicate detection")
group = feat_db.create_group("/", 'default', 'the default group')
raw_glob_feat_table = feat_db.create_table(group, 'raw_glob_feat', RawGlobFeature, 'Raw global features of images')


In [ ]:
inc_pca = IncrementalPCA(n_components=24)

In [ ]:
n = len(img_list)
bsize = 256
row = img_glob_feat_table.row
glob_feat_avg = np.zeros(116)
for i in range(math.ceil(n/bsize)):
    batch = []
    for p in img_list[i*bsize:min((i+1)*bsize, n)]:
        try:
            gray_img = cv2.cvtColor(cv2.imread(p), cv2.COLOR_BGR2GRAY)
            feat0 = edge_distribution(gray_img, (2,2), 12)
            feat1 = gray_averages(gray_img, (8,8))
            feat = feat0 + feat1
            batch += [feat]
            
            glob_feat_avg += feat
            j = p.rfind('/')
            row['hash'] = p[j+1:j+33]
            row['glob_feat'] = feat
            row.append()
        except Exception as e:
            print('Discard the image due to: %s' % str(e))
    if len(batch) < bsize:
        continue
    batch = np.array(batch)
    print(batch.shape)
    inc_pca.partial_fit(batch)

glob_feat_avg /= n
row['hash'] = 'avgxxx'
row['glob_feat'] = glob_feat_avg
row.append()

raw_glob_feat_table.flush()
feat_db.close()

# Apply PCA on raw lobal features

In [ ]:
class PCAedGlobFeature(IsDescription):
    hash = StringCol(48)
    pcaed_glob_feat  = Float64Col(24)

feat_db = tables.open_file("./dup_img_feat.h5", mode="a", title="Features for duplicate detection")
raw_glob_feat_table = feat_db.root.group.raw_glob_feat_table
pcaed_glob_feat_table = feat_db.create_table(feat_db.root.group, 'pcaed_glob_feat', PCAedGlobFeature, 'PCAed global features of images')


In [ ]:
bucket_thld = raw_glob_feat_table.where('hash==b"avgxxx"')  #Warning: Use b prefix.

In [ ]:
row = pcaed_glob_feat_table.row
for r in raw_glob_feat_table.iterrows():
    feat = r['glob_feat']
    f = inc_pca.transform(feat)
    row['hash'] = r['hash']
    row['pcaed_glob_feat'] = f
    row.append()
    
pcaed_glob_feat_table.flush()
feat_db.close()

# Clustering

In [ ]:
def vec_dist(f0, f1):
    return 0

def hamming_dist(sig0, sig1):
    return np.sum(sig0 != sig1)



In [ ]:
feat_db = tables.open_file("./dup_img_feat.h5", mode="r", title="Features for duplicate detection")
raw_glob_feat_table = feat_db.root.group.raw_glob_feat_table
pcaed_glob_feat_table = feat_db.root.group.pcaed_glob_feat_table

In [ ]:
clusters = {} # cluster_name: [hash1, hash2]
buckets = {} # bucket_sig: {points:[hash1, hash2], clusters:[cluster_name1]}
points = {} # hash: {cluster: cluster_name, bucket: bucket_sig, glob_feat: []}

eps = 1e-3

# n = pcaed_glob_feat_table.nrows
for r in pcaed_glob_feat_table.iterrows():
    h = r['hash']
    points[h] = {}
    p = points[h]
    p['cluster'] = 'c-' + h
    clusters[p['cluster']] = [h]
    
    p['glob_feat'] = r['pcaed_glob_feat']
    p['bucket'] = (np.cap['glob_feat'] > glob_feat_avg).astype(np.int32)
    
    if p['bucket'] in buckets:
        buckets[p['bucket']]['points'] += [h]
        buckets[p['bucket']]['clusters'] += [p['cluster']]
    else:
        buckets[p['bucket']] = {}
        buckets[p['bucket']]['points'] = [h]
        buckets[p['bucket']]['clusters'] = [p['cluster']]
    
    

# Clustering within each bucket
for _, buck in buckets:
    hashes = buck['points']
    n = len(hashes)
    for i in range(n):
        hi = hashes[i]
        pi = points[hi]
        for j in range(i+1,n):
            hj = hashes[j]
            pj = points[hj]
            d = vec_dist(pi['glob_feat'], pj['glob_feat'])  #TODO
            if d < eps:  # close enough, merge cj to ci.
                pj['cluster'] = pi['cluster']
                clusters[pi['cluster']] = clusters[pi['cluster']] + clusters[pj['cluster']]
                # Remove cj.
                del clusters[pj['cluster']]
                buck['clusters'].remove(pj['cluster'])
        

# Merge clusters across buckets
buk_sigs = buckets.keys()
for i, sig0 in enumerate(buk_sigs):
    for _, sig1 in enumerate(buk_sigs[i+1:]):
        diff = hamming_dist(sig0, sig1)
        if diff > 2: # Too far away and do not merge
            continue
        for c0_name in buckets[sig0]['clusters']:
            for c1_name in buckets[sig1]['clusters']:
                c0_hashes = clusters[c0_name]
                c1_hashes = clusters[c1_name]
                h0 = c0_hashes[np.random.randint(len(c0_hashes))]
                h1 = c1_hashes[np.random.randint(len(c1_hashes))]
                if vec_dist(points[h0]['glob_feat'], points[h1]['glob_feat']) < eps: # close enough, merge c1 to c0
                    for hh in c1_hashes:
                        points[hh]['cluster'] = c0_name
                        points[hh]['bucket'] = sig0
                        buckets[sig1]['points'].remove[hh]
                    clusters[c0_name] = clusters[c0_name] + clusters[c1_name]
                    del clusters[c1_name]
                    buckets[sig1]['clusters'].remove(c1)
            
            # The bucket sig1 is empty and removed.
            if len(buckets[sig1]['points']) == 0 and len(buckets[sig1]['clusters']) == 0:
                del buckets[sig1]
        
